In [7]:
!pip install transformers==3.5.1
!pip install nlp==0.4.0

Looking in indexes: http://ftp.daumkakao.com/pypi/simple
     |████████████████████████████████| 1.3 MB 20.3 MB/s            
     |████████████████████████████████| 2.9 MB 102.4 MB/s            
     |████████████████████████████████| 1.1 MB 106.5 MB/s            
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.8.0rc4
    Uninstalling tokenizers-0.8.0rc4:
      Successfully uninstalled tokenizers-0.8.0rc4
  Attempting uninstall: sentencepiece
    Found existing installation: sentencepiece 0.1.96
    Uninstalling sentencepiece-0.1.96:
      Successfully uninstalled sentencepiece-0.1.96
  Attempting uninstall: transformers
    Found existing installation: transformers 3.0.0
    Uninstalling transformers-3.0.0:
      Successfully uninstalled transformers-3.0.0
You should consider upgrading via the '/home/ubuntu/anaconda3/envs/pytorch_p36/bin/python -m pip install --upgrade pip' command.


In [98]:
import transformers

In [99]:
transformers.__version__

'3.5.1'

In [100]:
import torch

In [101]:
torch.__version__

'1.7.1+cu101'

In [102]:
!nvidia-smi

Thu Nov 11 16:24:14 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 418.67       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  On   | 00000000:00:05.0 Off |                  Off |
| N/A   40C    P0    64W / 300W |   9535MiB / 32480MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

### 사전학습된 모델을 불러온다.
- https://huggingface.co/transformers

In [103]:
from transformers import BertModel, BertTokenizer
import torch

In [104]:
model = BertModel.from_pretrained('bert-base-uncased')

In [105]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

### preprocessing

In [106]:
sentence = 'I love Paris'
tokens = tokenizer.tokenize(sentence)

In [107]:
tokens = ['[CLS]'] + tokens + ['[SEP]']

- PAD 를 해주어야 하는데 여기서는 길이가 10인 것을 가정함.
- 일반적으로는 max_length를 기준으로 sequence length를 맞춰줌.

In [108]:
while True:
    tokens += ['[PAD]']
    if len(tokens) == 10 :
        break

In [109]:
tokens

['[CLS]',
 'i',
 'love',
 'paris',
 '[SEP]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]']

- attention_mask 를 만들어준다.

In [110]:
attention_mask = [1 if x!= '[PAD]' else 0 for x in tokens]

In [111]:
tokens_ids = tokenizer.convert_tokens_to_ids(tokens)

- 고유한 tokens의 ID가 매겨진다

In [112]:
tokens_ids

[101, 1045, 2293, 3000, 102, 0, 0, 0, 0, 0]

In [113]:
tokens_ids = torch.tensor(tokens_ids).unsqueeze(0)

In [114]:
attention_mask = torch.tensor(attention_mask).unsqueeze(0)

- hidden_rep : 입력에 대한 모든 토큰의 임베딩
- clas_head : [CLS] 토큰의 표현 [CLS] 토큰은 문장 전체의 표현을 보유하고 있음.

In [115]:
hidden_rep, cls_head = model(tokens_ids, attention_mask =  attention_mask)

- torch.Size(batch_size, sequence_length, hidden_size)

In [116]:
hidden_rep.shape

torch.Size([1, 10, 768])

### extract embedding

In [117]:
model = BertModel.from_pretrained('bert-base-uncased', 
                                 output_hidden_states = True)
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [118]:
sentence = 'I love paris.'

tokens = tokenizer.tokenize(sentence)

tokens = ['[CLS]'] + tokens + ['[SEP]']

while True:
    tokens += ['[PAD]']
    if len(tokens) == 10 :
        break

attention_mask = [1 if x!= '[PAD]' else 0 for x in tokens]

tokens_ids = tokenizer.convert_tokens_to_ids(tokens)

In [119]:
tokens_ids = torch.tensor(tokens_ids).unsqueeze(0)
attention_mask = torch.tensor(attention_mask).unsqueeze(0)

- output_hidden_states = True 로 지정해주었기 때문에 model의 output은 3개가 나온다
- last_hidden_state : 최종 인코더 계층 (12번째 인코더)에서만 얻은 모든 토큰의 표현을 가진다.
- pooler_output : 최종 인코더 계층의 [CLS] 토큰 표현을 나타내며 선형 및 tanh 활성화 함수에 의해 계산된다.
- hidden_states : 모든 인코더 계층에서 얻은 모든 토큰의 표현을 포함한다.

In [120]:
last_hidden_state, pooler_output, hidden_states = model(tokens_ids, attention_mask = attention_mask)

In [121]:
last_hidden_state

tensor([[[-0.0296,  0.3773, -0.1839,  ..., -0.5360,  0.4397,  0.5657],
         [ 0.3598,  0.6302, -0.4997,  ..., -0.1223,  0.9489,  0.4497],
         [ 1.1474,  0.7872,  0.6734,  ..., -0.7419,  0.8205,  0.2102],
         ...,
         [ 0.1189,  0.1819,  0.1520,  ..., -0.1528,  0.1360,  0.1827],
         [ 0.2906,  0.1294,  0.2761,  ..., -0.2681,  0.0115,  0.2013],
         [-0.0191, -0.0996,  0.0862,  ..., -0.0082,  0.1182, -0.0335]]],
       grad_fn=<NativeLayerNormBackward>)

In [54]:
last_hidden_state.shape

torch.Size([1, 10, 768])

In [56]:
pooler_output.shape

torch.Size([1, 768])

In [58]:
len(hidden_states)

13

- hidden_states[0]는 입력 임베딩 레이어 $h_0$ 에서 얻은 모든 토큰의 표현 벡터를 가짐
- hidden_states[1]는 입력 임베딩 레이어 $h_1$ 에서 얻은 모든 토큰의 표현 벡터를 가짐
- hidden_states[2]는 입력 임베딩 레이어 $h_2$ 에서 얻은 모든 토큰의 표현 벡터를 가짐
- hidden_states[12]는 입력 임베딩 레이어 $h_{12}$ 에서 얻은 모든 토큰의 표현 벡터를 가짐

### Downstream
- text classification

label과 함께 문장이 포함된 데이터 셋이 있고 가정하자. 위 과정과 동일하게 ($[CLS]$, $[SEP]$ 추가하는 것 까지) 한 후 모든 토큰을 무시하
고 $R_{[CLS]}$ 를 추출한 후 $R_{[CLS]}$를 분류기(feed forward)에 입력하고 학습시켜 감정 분석을 수행한다.

Downstream을 하는 경우에 총 두가지 방법이 존재한다. 분류 계층과 함께 사전 학습된 BERT 모델의 가중치를 업데이트 하는 방법과, 사전 학습된 BERT모델이 아닌 분류 계층의 가중치만 업데이트하는 방법이다.

In [122]:
from transformers import BertForSequenceClassification, BertTokenizerFast, Trainer, TrainingArguments 
from nlp import load_dataset 
import torch
import numpy as np 

### 데이터 셋과 모델 로딩하기

In [123]:
# !gdown https://drive.google.com/uc?id=11_M4ootuT7I1G0RlihcC0cA3Elqotlc-
dataset = load_dataset('csv', data_files='./imdbs.csv', split='train')

Using custom data configuration default


In [127]:
dataset = dataset.train_test_split(test_size = 0.3)

In [128]:
dataset

{'train': Dataset(features: {'text': Value(dtype='string', id=None), 'label': Value(dtype='int64', id=None)}, num_rows: 28000),
 'test': Dataset(features: {'text': Value(dtype='string', id=None), 'label': Value(dtype='int64', id=None)}, num_rows: 12000)}

In [129]:
train_set = dataset['train']
test_set = dataset['test']

In [130]:
model = BertForSequenceClassification.from_pretrained('bert-base-uncased').to('cuda:0')

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

여기서는 BertTokenizer 대신 BertTokenizerFast를 사용했다. 

In [131]:
tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')

토크나이저를 사용하지 않으면 위 경우처럼 각각 attention_mask 와 pad 등을 지정해주어야한다.

In [132]:
tokenizer(['I love Paris', 
          'birds fly', 
          'snow fall'], padding = True, max_length = 5)

{'input_ids': [[101, 1045, 2293, 3000, 102], [101, 5055, 4875, 102, 0], [101, 4586, 2991, 102, 0]], 'token_type_ids': [[0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0]], 'attention_mask': [[1, 1, 1, 1, 1], [1, 1, 1, 1, 0], [1, 1, 1, 1, 0]]}

In [133]:
def preprocess(data) :
    return tokenizer(data['text'], padding = True, truncation = True)

In [134]:
train_set = train_set.map(preprocess, batched = True, batch_size = len(train_set))
test_set = test_set.map(preprocess, batched = True, batch_size = len(train_set))

In [136]:
train_set.set_format('torch', columns = ['input_ids', 'attention_mask', 'label'])
test_set.set_format('torch', columns = ['input_ids', 'attention_mask', 'label'])

### 모델 학습

In [137]:
batch_size = 8
epochs = 2

warmup_steps = 500
weight_decay = 0.01

### 학습 인수 정하기

In [138]:
training_args = TrainingArguments(
output_dir = '.results', 
num_train_epochs = epochs, 
per_device_train_batch_size = batch_size, 
per_device_eval_batch_size = batch_size, 
warmup_steps = warmup_steps, 
weight_decay = weight_decay, 
evaluate_during_training = True, 
logging_dir = './logs',
)

/home/ubuntu/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/transformers/training_args.py:347: FutureWarning: The `evaluate_during_training` argument is deprecated in favor of `evaluation_strategy` (which has more options)
  FutureWarning,


In [139]:
trainer = Trainer(
model=model,
args = training_args, 
train_dataset = train_set, 
eval_dataset=test_set)

In [140]:
trainer.train()

Step,Training Loss,Validation Loss
500,0.452546,0.336160
1000,0.357040,0.318635
1500,0.344358,0.360417
2000,0.287199,0.251012
2500,0.280747,0.302461
3000,0.270211,0.296378
3500,0.274113,0.310389
4000,0.175322,0.310107
4500,0.151284,0.291164
5000,0.152887,0.325716


TrainOutput(global_step=7000, training_loss=0.23955782645089285)

In [141]:
trainer.evaluate()

{'eval_loss': 0.29067906737327576, 'epoch': 2.0}

### QnA

In [145]:
from transformers import BertForQuestionAnswering, BertTokenizer

In [176]:
model = BertForQuestionAnswering.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')
tokenizer = BertTokenizer.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')

In [177]:
question = '면역 체계는 무엇입니까?'

paragraph = '면역 체계는 질병으로부터 보호하는 유기체 내의 다양한 생물학적 구조와 과정의 시스템입니다. \
제대로 기능하려면 면역 체계가 바이러스에서 기생충에 이르기까지 병원균으로 알려진 다양한 물질을 탐지하고 유기체의 \
건강한 조직과 구별해야 합니다.'

In [178]:
question = '[CLS] ' + question + '[SEP]'
paragraph = paragraph + '[SEP]'

In [179]:
question_tokens = tokenizer.tokenize(question)
paragraph_tokens = tokenizer.tokenize(paragraph)

In [180]:
tokens = question_tokens + paragraph_tokens
input_ids = tokenizer.convert_tokens_to_ids(tokens)

In [181]:
segment_ids = [0] * len(question_tokens)
segment_ids += [1] * len(paragraph_tokens)

In [182]:
input_ids = torch.tensor([input_ids])
segment_ids = torch.tensor([segment_ids])

모든 토큰에 대한 시작 점수와 끝 점수를 반환하는 모델(Q&A fine tuning된 BERT 모델)

In [155]:
start_scores, end_scores = model(input_ids, token_type_ids = segment_ids)

시작 점수와 끝 점수를 반환하는데, 각 인덱스를 가지고와 질응 답변을 가지고 옵니다. 

In [158]:
start_idx = torch.argmax(start_scores)
end_idx = torch.argmax(end_scores)

print(''.join(tokens[start_idx:end_idx+1]))

ᄌ##ᅵ##ᆯ##ᄇ##ᅧ##ᆼ##ᄋ##ᅳ##ᄅ##ᅩ##ᄇ##ᅮ##ᄐ##ᅥᄇ##ᅩ##ᄒ##ᅩ##ᄒ##ᅡ##ᄂ##ᅳ##ᆫᄋ##ᅲ##ᄀ##ᅵ##ᄎ##ᅦᄂ##ᅢ##ᄋ##ᅴ[UNK]ᄉ##ᅢ##ᆼ##ᄆ##ᅮ##ᆯ##ᄒ##ᅡ##ᆨ##ᄌ##ᅥ##ᆨᄀ##ᅮ##ᄌ##ᅩ##ᄋ##ᅪᄀ##ᅪ##ᄌ##ᅥ##ᆼ##ᄋ##ᅴᄉ##ᅵ##ᄉ##ᅳ##ᄐ##ᅦ##ᆷ##ᄋ##ᅵ##ᆸ##ᄂ##ᅵ##ᄃ##ᅡ
